# Stock purchase recommendations with Machine Learning

In [37]:
import pandas as pd
import numpy as np
import talib as ta
import matplotlib.pyplot as plt
from tqdm import tqdm, tqdm_notebook # progress bar
import fastparquet

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier

In [38]:
pd.set_option('display.max_columns', 1500)

## Machine Learning

In [39]:
# load the df_Xy dataframe from feature engineering step:
df_Xy = fastparquet.ParquetFile('../data/interim/df_Xy_all_tickers_final.parq').to_pandas()

# split df into features and target (column setup_for_profitable_trade)
df_X = df_Xy.drop(columns=['setup_for_profitable_trade'])
df_y = df_Xy['setup_for_profitable_trade']

df_X.shape

(8776, 693)

### Build Model

In [40]:
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, shuffle=False)

In [55]:
X_train.tail()

,AdjVolume_-19,AdjVolume_-18,AdjVolume_-17,AdjVolume_-16,AdjVolume_-15,AdjVolume_-14,AdjVolume_-13,AdjVolume_-12,AdjVolume_-11,AdjVolume_-10,AdjVolume_-9,AdjVolume_-8,AdjVolume_-7,AdjVolume_-6,AdjVolume_-5,AdjVolume_-4,AdjVolume_-3,AdjVolume_-2,AdjVolume_-1,AdjVolume_0,AdjOpen_-19,AdjOpen_-18,AdjOpen_-17,AdjOpen_-16,AdjOpen_-15,AdjOpen_-14,AdjOpen_-13,AdjOpen_-12,AdjOpen_-11,AdjOpen_-10,AdjOpen_-9,AdjOpen_-8,AdjOpen_-7,AdjOpen_-6,AdjOpen_-5,AdjOpen_-4,AdjOpen_-3,AdjOpen_-2,AdjOpen_-1,AdjOpen_0,AdjHigh_-19,AdjHigh_-18,AdjHigh_-17,AdjHigh_-16,AdjHigh_-15,AdjHigh_-14,AdjHigh_-13,AdjHigh_-12,AdjHigh_-11,AdjHigh_-10,AdjHigh_-9,AdjHigh_-8,AdjHigh_-7,AdjHigh_-6,AdjHigh_-5,AdjHigh_-4,AdjHigh_-3,AdjHigh_-2,AdjHigh_-1,AdjHigh_0,AdjLow_-19,AdjLow_-18,AdjLow_-17,AdjLow_-16,AdjLow_-15,AdjLow_-14,AdjLow_-13,AdjLow_-12,AdjLow_-11,AdjLow_-10,AdjLow_-9,AdjLow_-8,AdjLow_-7,AdjLow_-6,AdjLow_-5,AdjLow_-4,AdjLow_-3,AdjLow_-2,AdjLow_-1,AdjLow_0,AdjClose_-19,AdjClose_-18,AdjClose_-17,AdjClose_-16,AdjClose_-15,AdjClose_-14,AdjClose_-13,AdjClose_-12,AdjClose_-11,AdjClose_-10,AdjClose_-9,AdjClose_-8,AdjClose_-7,AdjClose_-6,AdjClose_-5,AdjClose_-4,AdjClose_-3,AdjClose_-2,AdjClose_-1,AdjClose_0,AdjCloseRSI14_-19,AdjCloseRSI14_-18,AdjCloseRSI14_-17,AdjCloseRSI14_-16,AdjCloseRSI14_-15,AdjCloseRSI14_-14,AdjCloseRSI14_-13,AdjCloseRSI14_-12,AdjCloseRSI14_-11,AdjCloseRSI14_-10,AdjCloseRSI14_-9,AdjCloseRSI14_-8,AdjCloseRSI14_-7,AdjCloseRSI14_-6,AdjCloseRSI14_-5,AdjCloseRSI14_-4,AdjCloseRSI14_-3,AdjCloseRSI14_-2,AdjCloseRSI14_-1,AdjCloseRSI14_0,RSI_above_80_-19,RSI_above_80_-18,RSI_above_80_-17,RSI_above_80_-16,RSI_above_80_-15,RSI_above_80_-14,RSI_above_80_-13,RSI_above_80_-12,RSI_above_80_-11,RSI_above_80_-10,RSI_above_80_-9,RSI_above_80_-8,RSI_above_80_-7,RSI_above_80_-6,RSI_above_80_-5,RSI_above_80_-4,RSI_above_80_-3,RSI_above_80_-2,RSI_above_80_-1,RSI_above_80_0,RSI_below_20_-19,RSI_below_20_-18,RSI_below_20_-17,RSI_below_20_-16,RSI_below_20_-15,RSI_below_20_-14,RSI_below_20_-13,RSI_below_20_-12,RSI_below_20_-11,RSI_below_20_-10,RSI_below_20_-9,RSI_below_20_-8,RSI_below_20_-7,RSI_below_20_-6,RSI_below_20_-5,RSI_below_20_-4,RSI_below_20_-3,RSI_below_20_-2,RSI_below_20_-1,RSI_below_20_0,AdjCloseSMA10_-19,AdjCloseSMA10_-18,AdjCloseSMA10_-17,AdjCloseSMA10_-16,AdjCloseSMA10_-15,AdjCloseSMA10_-14,AdjCloseSMA10_-13,AdjCloseSMA10_-12,AdjCloseSMA10_-11,AdjCloseSMA10_-10,AdjCloseSMA10_-9,AdjCloseSMA10_-8,AdjCloseSMA10_-7,AdjCloseSMA10_-6,AdjCloseSMA10_-5,AdjCloseSMA10_-4,AdjCloseSMA10_-3,AdjCloseSMA10_-2,AdjCloseSMA10_-1,AdjCloseSMA10_0,AdjCloseSMA10_to_AdjClose_ratio_-19,AdjCloseSMA10_to_AdjClose_ratio_-18,AdjCloseSMA10_to_AdjClose_ratio_-17,AdjCloseSMA10_to_AdjClose_ratio_-16,AdjCloseSMA10_to_AdjClose_ratio_-15,AdjCloseSMA10_to_AdjClose_ratio_-14,AdjCloseSMA10_to_AdjClose_ratio_-13,AdjCloseSMA10_to_AdjClose_ratio_-12,AdjCloseSMA10_to_AdjClose_ratio_-11,AdjCloseSMA10_to_AdjClose_ratio_-10,AdjCloseSMA10_to_AdjClose_ratio_-9,AdjCloseSMA10_to_AdjClose_ratio_-8,AdjCloseSMA10_to_AdjClose_ratio_-7,AdjCloseSMA10_to_AdjClose_ratio_-6,AdjCloseSMA10_to_AdjClose_ratio_-5,AdjCloseSMA10_to_AdjClose_ratio_-4,AdjCloseSMA10_to_AdjClose_ratio_-3,AdjCloseSMA10_to_AdjClose_ratio_-2,AdjCloseSMA10_to_AdjClose_ratio_-1,AdjCloseSMA10_to_AdjClose_ratio_0,AdjClose_above_AdjCloseSMA10_-19,AdjClose_above_AdjCloseSMA10_-18,AdjClose_above_AdjCloseSMA10_-17,AdjClose_above_AdjCloseSMA10_-16,AdjClose_above_AdjCloseSMA10_-15,AdjClose_above_AdjCloseSMA10_-14,AdjClose_above_AdjCloseSMA10_-13,AdjClose_above_AdjCloseSMA10_-12,AdjClose_above_AdjCloseSMA10_-11,AdjClose_above_AdjCloseSMA10_-10,AdjClose_above_AdjCloseSMA10_-9,AdjClose_above_AdjCloseSMA10_-8,AdjClose_above_AdjCloseSMA10_-7,AdjClose_above_AdjCloseSMA10_-6,AdjClose_above_AdjCloseSMA10_-5,AdjClose_above_AdjCloseSMA10_-4,AdjClose_above_AdjCloseSMA10_-3,AdjClose_above_AdjCloseSMA10_-2,AdjClose_above_AdjCloseSMA10_-1,AdjClose_above_AdjCloseSMA10_0,AdjClose_crossed_AdjCloseSMA10_-19,AdjClose_crossed_AdjCloseSMA10_-18,AdjClose_crossed_AdjCloseSMA10

In [56]:
X_test.head()

,AdjVolume_-19,AdjVolume_-18,AdjVolume_-17,AdjVolume_-16,AdjVolume_-15,AdjVolume_-14,AdjVolume_-13,AdjVolume_-12,AdjVolume_-11,AdjVolume_-10,AdjVolume_-9,AdjVolume_-8,AdjVolume_-7,AdjVolume_-6,AdjVolume_-5,AdjVolume_-4,AdjVolume_-3,AdjVolume_-2,AdjVolume_-1,AdjVolume_0,AdjOpen_-19,AdjOpen_-18,AdjOpen_-17,AdjOpen_-16,AdjOpen_-15,AdjOpen_-14,AdjOpen_-13,AdjOpen_-12,AdjOpen_-11,AdjOpen_-10,AdjOpen_-9,AdjOpen_-8,AdjOpen_-7,AdjOpen_-6,AdjOpen_-5,AdjOpen_-4,AdjOpen_-3,AdjOpen_-2,AdjOpen_-1,AdjOpen_0,AdjHigh_-19,AdjHigh_-18,AdjHigh_-17,AdjHigh_-16,AdjHigh_-15,AdjHigh_-14,AdjHigh_-13,AdjHigh_-12,AdjHigh_-11,AdjHigh_-10,AdjHigh_-9,AdjHigh_-8,AdjHigh_-7,AdjHigh_-6,AdjHigh_-5,AdjHigh_-4,AdjHigh_-3,AdjHigh_-2,AdjHigh_-1,AdjHigh_0,AdjLow_-19,AdjLow_-18,AdjLow_-17,AdjLow_-16,AdjLow_-15,AdjLow_-14,AdjLow_-13,AdjLow_-12,AdjLow_-11,AdjLow_-10,AdjLow_-9,AdjLow_-8,AdjLow_-7,AdjLow_-6,AdjLow_-5,AdjLow_-4,AdjLow_-3,AdjLow_-2,AdjLow_-1,AdjLow_0,AdjClose_-19,AdjClose_-18,AdjClose_-17,AdjClose_-16,AdjClose_-15,AdjClose_-14,AdjClose_-13,AdjClose_-12,AdjClose_-11,AdjClose_-10,AdjClose_-9,AdjClose_-8,AdjClose_-7,AdjClose_-6,AdjClose_-5,AdjClose_-4,AdjClose_-3,AdjClose_-2,AdjClose_-1,AdjClose_0,AdjCloseRSI14_-19,AdjCloseRSI14_-18,AdjCloseRSI14_-17,AdjCloseRSI14_-16,AdjCloseRSI14_-15,AdjCloseRSI14_-14,AdjCloseRSI14_-13,AdjCloseRSI14_-12,AdjCloseRSI14_-11,AdjCloseRSI14_-10,AdjCloseRSI14_-9,AdjCloseRSI14_-8,AdjCloseRSI14_-7,AdjCloseRSI14_-6,AdjCloseRSI14_-5,AdjCloseRSI14_-4,AdjCloseRSI14_-3,AdjCloseRSI14_-2,AdjCloseRSI14_-1,AdjCloseRSI14_0,RSI_above_80_-19,RSI_above_80_-18,RSI_above_80_-17,RSI_above_80_-16,RSI_above_80_-15,RSI_above_80_-14,RSI_above_80_-13,RSI_above_80_-12,RSI_above_80_-11,RSI_above_80_-10,RSI_above_80_-9,RSI_above_80_-8,RSI_above_80_-7,RSI_above_80_-6,RSI_above_80_-5,RSI_above_80_-4,RSI_above_80_-3,RSI_above_80_-2,RSI_above_80_-1,RSI_above_80_0,RSI_below_20_-19,RSI_below_20_-18,RSI_below_20_-17,RSI_below_20_-16,RSI_below_20_-15,RSI_below_20_-14,RSI_below_20_-13,RSI_below_20_-12,RSI_below_20_-11,RSI_below_20_-10,RSI_below_20_-9,RSI_below_20_-8,RSI_below_20_-7,RSI_below_20_-6,RSI_below_20_-5,RSI_below_20_-4,RSI_below_20_-3,RSI_below_20_-2,RSI_below_20_-1,RSI_below_20_0,AdjCloseSMA10_-19,AdjCloseSMA10_-18,AdjCloseSMA10_-17,AdjCloseSMA10_-16,AdjCloseSMA10_-15,AdjCloseSMA10_-14,AdjCloseSMA10_-13,AdjCloseSMA10_-12,AdjCloseSMA10_-11,AdjCloseSMA10_-10,AdjCloseSMA10_-9,AdjCloseSMA10_-8,AdjCloseSMA10_-7,AdjCloseSMA10_-6,AdjCloseSMA10_-5,AdjCloseSMA10_-4,AdjCloseSMA10_-3,AdjCloseSMA10_-2,AdjCloseSMA10_-1,AdjCloseSMA10_0,AdjCloseSMA10_to_AdjClose_ratio_-19,AdjCloseSMA10_to_AdjClose_ratio_-18,AdjCloseSMA10_to_AdjClose_ratio_-17,AdjCloseSMA10_to_AdjClose_ratio_-16,AdjCloseSMA10_to_AdjClose_ratio_-15,AdjCloseSMA10_to_AdjClose_ratio_-14,AdjCloseSMA10_to_AdjClose_ratio_-13,AdjCloseSMA10_to_AdjClose_ratio_-12,AdjCloseSMA10_to_AdjClose_ratio_-11,AdjCloseSMA10_to_AdjClose_ratio_-10,AdjCloseSMA10_to_AdjClose_ratio_-9,AdjCloseSMA10_to_AdjClose_ratio_-8,AdjCloseSMA10_to_AdjClose_ratio_-7,AdjCloseSMA10_to_AdjClose_ratio_-6,AdjCloseSMA10_to_AdjClose_ratio_-5,AdjCloseSMA10_to_AdjClose_ratio_-4,AdjCloseSMA10_to_AdjClose_ratio_-3,AdjCloseSMA10_to_AdjClose_ratio_-2,AdjCloseSMA10_to_AdjClose_ratio_-1,AdjCloseSMA10_to_AdjClose_ratio_0,AdjClose_above_AdjCloseSMA10_-19,AdjClose_above_AdjCloseSMA10_-18,AdjClose_above_AdjCloseSMA10_-17,AdjClose_above_AdjCloseSMA10_-16,AdjClose_above_AdjCloseSMA10_-15,AdjClose_above_AdjCloseSMA10_-14,AdjClose_above_AdjCloseSMA10_-13,AdjClose_above_AdjCloseSMA10_-12,AdjClose_above_AdjCloseSMA10_-11,AdjClose_above_AdjCloseSMA10_-10,AdjClose_above_AdjCloseSMA10_-9,AdjClose_above_AdjCloseSMA10_-8,AdjClose_above_AdjCloseSMA10_-7,AdjClose_above_AdjCloseSMA10_-6,AdjClose_above_AdjCloseSMA10_-5,AdjClose_above_AdjCloseSMA10_-4,AdjClose_above_AdjCloseSMA10_-3,AdjClose_above_AdjCloseSMA10_-2,AdjClose_above_AdjCloseSMA10_-1,AdjClose_above_AdjCloseSMA10_0,AdjClose_crossed_AdjCloseSMA10_-19,AdjClose_crossed_AdjCloseSMA10_-18,AdjClose_crossed_AdjCloseSMA10

In [41]:
pipeline = Pipeline([
    ('standardScaler', StandardScaler()),
    ('randomForest', RandomForestClassifier())
])
pipeline.get_params()

{'memory': None,
 'randomForest': RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
             max_depth=None, max_features='auto', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=None,
             oob_score=False, random_state=None, verbose=0,
             warm_start=False),
 'randomForest__bootstrap': True,
 'randomForest__class_weight': None,
 'randomForest__criterion': 'gini',
 'randomForest__max_depth': None,
 'randomForest__max_features': 'auto',
 'randomForest__max_leaf_nodes': None,
 'randomForest__min_impurity_decrease': 0.0,
 'randomForest__min_impurity_split': None,
 'randomForest__min_samples_leaf': 1,
 'randomForest__min_samples_split': 2,
 'randomForest__min_weight_fraction_leaf': 0.0,
 'randomForest__n_estimators': 'warn',
 'randomForest__n_jobs': None,
 'randomForest__oob_score': F

In [42]:
pipeline.fit(X_train, y_train)

C:\Users\Lindstaedt\Anaconda3\envs\trader_env\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Lindstaedt\Anaconda3\envs\trader_env\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)
C:\Users\Lindstaedt\Anaconda3\envs\trader_env\lib\site-packages\sklearn\ensemble\forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
     steps=[('standardScaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomForest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [43]:
y_pred = pipeline.predict(X_test)

C:\Users\Lindstaedt\Anaconda3\envs\trader_env\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [44]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.80      0.76      0.78      1300
        True       0.40      0.46      0.43       456

   micro avg       0.68      0.68      0.68      1756
   macro avg       0.60      0.61      0.60      1756
weighted avg       0.70      0.68      0.69      1756



In [45]:
accuracy_score(y_test, y_pred)

0.6810933940774487

In [46]:
confusion_matrix(y_test, y_pred, labels=[False, True])

array([[988, 312],
       [248, 208]], dtype=int64)

In [47]:
def print_feature_importances(estimator):

    importances = estimator.feature_importances_
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X_train.shape[1]):
        print(str(f + 1) + " importance: " + str(importances[indices[f]]) + ". feature name: " + X_train.columns[indices[f]])
        
print_feature_importances(pipeline.named_steps['randomForest'])

Feature ranking:
1 importance: 0.016560403959184768. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-14
2 importance: 0.01568061987809842. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-4
3 importance: 0.011992731129428258. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-2
4 importance: 0.011965728683777044. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-17
5 importance: 0.011372867843665337. feature name: AdjCloseSMA50_-15
6 importance: 0.011286104204772128. feature name: AdjCloseSMA50_chg_-16
7 importance: 0.01090352268466895. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-15
8 importance: 0.01023985680442988. feature name: AdjCloseSMA200_-13
9 importance: 0.010107204886495871. feature name: AdjCloseSMA200_-12
10 importance: 0.01004055856082407. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-8
11 importance: 0.009505500002634849. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-12
12 importance: 0.009185756613432928. feature name: AdjCloseSMA50_t

106 importance: 0.003144358159339733. feature name: AdjCloseSMA50_to_AdjClose_ratio_-17
107 importance: 0.003142365074585482. feature name: AdjCloseSMA200_to_AdjClose_ratio_-6
108 importance: 0.003135106007655733. feature name: AdjOpen_-11
109 importance: 0.00312473477587904. feature name: AdjClose_above_AdjCloseSMA10_-15
110 importance: 0.0031231143862675997. feature name: AdjCloseSMA10_-1
111 importance: 0.003119289751342076. feature name: AdjCloseSMA50_to_AdjClose_ratio_-8
112 importance: 0.0030802084952964567. feature name: AdjCloseSMA10_to_AdjClose_ratio_-19
113 importance: 0.003078570485891123. feature name: AdjOpen_-7
114 importance: 0.003064878929975666. feature name: AdjCloseSMA50_-8
115 importance: 0.0030329693868196847. feature name: AdjCloseSMA10_to_AdjCloseSMA50_ratio_0
116 importance: 0.0030315488361522805. feature name: AdjCloseSMA10_to_AdjClose_ratio_-7
117 importance: 0.003003151109804415. feature name: AdjHigh_chg_-10
118 importance: 0.0029984364449459565. feature nam

216 importance: 0.0014424866662507898. feature name: AdjCloseSMA50_to_AdjClose_ratio_-14
217 importance: 0.0014216227644170986. feature name: AdjHigh_chg_-11
218 importance: 0.001413503169070135. feature name: AdjVolume_-9
219 importance: 0.001377865624041396. feature name: AdjCloseSMA10_to_AdjClose_ratio_-3
220 importance: 0.0013662786232473121. feature name: AdjOpen_-14
221 importance: 0.0013611577661130415. feature name: AdjCloseSMA10_-5
222 importance: 0.0013592408879470655. feature name: AdjLow_chg_-15
223 importance: 0.0013437659036925544. feature name: AdjClose_crossed_AdjCloseSMA50_-8
224 importance: 0.001337367406969389. feature name: AdjLow_chg_-6
225 importance: 0.001334073329206808. feature name: AdjCloseSMA10_chg_-13
226 importance: 0.001323512185024232. feature name: AdjHigh_-4
227 importance: 0.0013209928227224724. feature name: AdjCloseSMA200_-7
228 importance: 0.0013060403968403055. feature name: AdjHigh_chg_-19
229 importance: 0.001299832868099198. feature name: AdjVo

329 importance: 0.00048519839168889264. feature name: AdjHigh_-9
330 importance: 0.0004805216489296481. feature name: AdjCloseSMA50_above_AdjCloseSMA10_-18
331 importance: 0.00047575758602031603. feature name: AdjCloseSMA10_to_AdjClose_ratio_-13
332 importance: 0.0004731048016525491. feature name: AdjCloseRSI14_-17
333 importance: 0.0004700640815073374. feature name: AdjOpen_-15
334 importance: 0.0004687094352878808. feature name: AdjCloseSMA10_chg_-3
335 importance: 0.0004645571837999372. feature name: AdjVolume_-10
336 importance: 0.0004619010378658927. feature name: AdjCloseSMA50_chg_-1
337 importance: 0.0004511708671301617. feature name: AdjCloseSMA10_to_AdjClose_ratio_-16
338 importance: 0.00045007356971812706. feature name: AdjCloseRSI14_-4
339 importance: 0.00044989508138704317. feature name: AdjCloseSMA50_crossed_AdjCloseSMA10_-12
340 importance: 0.0004447078155140593. feature name: AdjCloseSMA50_to_AdjClose_ratio_-15
341 importance: 0.0004391028519823063. feature name: AdjLow_

438 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA50_-2
439 importance: 0.0. feature name: AdjCloseSMA200_crossed_AdjCloseSMA50_-16
440 importance: 0.0. feature name: AdjCloseRSI14_-11
441 importance: 0.0. feature name: RSI_above_80_-12
442 importance: 0.0. feature name: RSI_above_80_-10
443 importance: 0.0. feature name: RSI_above_80_-9
444 importance: 0.0. feature name: RSI_above_80_-8
445 importance: 0.0. feature name: RSI_above_80_-7
446 importance: 0.0. feature name: RSI_above_80_-6
447 importance: 0.0. feature name: RSI_above_80_-5
448 importance: 0.0. feature name: RSI_above_80_-4
449 importance: 0.0. feature name: RSI_above_80_-3
450 importance: 0.0. feature name: RSI_above_80_-2
451 importance: 0.0. feature name: RSI_above_80_-1
452 importance: 0.0. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-5
453 importance: 0.0. feature name: RSI_below_20_-19
454 importance: 0.0. feature name: RSI_below_20_-18
455 importance: 0.0. feature name: RSI_below_20_-17
456 importan

576 importance: 0.0. feature name: AdjClose_crossed_AdjCloseSMA10_-1
577 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-5
578 importance: 0.0. feature name: AdjClose_crossed_AdjCloseSMA10_-7
579 importance: 0.0. feature name: AdjClose_crossed_AdjCloseSMA10_-5
580 importance: 0.0. feature name: AdjClose_crossed_AdjCloseSMA10_-4
581 importance: 0.0. feature name: AdjClose_crossed_AdjCloseSMA10_-3
582 importance: 0.0. feature name: AdjClose_crossed_AdjCloseSMA10_-2
583 importance: 0.0. feature name: MSFT.US
584 importance: 0.0. feature name: AdjCloseSMA50_0
585 importance: 0.0. feature name: RSI_below_20_-12
586 importance: 0.0. feature name: AdjCloseSMA200_0
587 importance: 0.0. feature name: AdjClose_crossed_AdjCloseSMA50_-19
588 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA50_-18
589 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA50_-17
590 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA50_-16
591 importance: 0.0. feature name: AdjClose_ab

### Improve RandomForest model

In [48]:
parameters = {
    'randomForest__min_samples_leaf': [1, 2, 5, 10]
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=5, scoring='f1', n_jobs=-1, verbose=6)
cv.fit(X_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   28.7s
[Parallel(n_jobs=-1)]: Done  17 out of  20 | elapsed:  1.2min remaining:   12.1s
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:  1.2min finished
C:\Users\Lindstaedt\Anaconda3\envs\trader_env\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Lindstaedt\Anaconda3\envs\trader_env\lib\site-packages\sklearn\base.py:465: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('standardScaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('randomForest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'randomForest__min_samples_leaf': [1, 2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=6)

In [49]:
cv.best_params_

{'randomForest__min_samples_leaf': 2}

In [50]:
print_feature_importances(cv.best_estimator_.named_steps['randomForest'])

Feature ranking:
1 importance: 0.0195709463040589. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-15
2 importance: 0.015165215992323672. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-1
3 importance: 0.014677313740682475. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-3
4 importance: 0.013826877835253094. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-2
5 importance: 0.013518114299001502. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-16
6 importance: 0.013450591238048254. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-18
7 importance: 0.011381910611114873. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_-19
8 importance: 0.010971447086976666. feature name: AdjCloseSMA50_-16
9 importance: 0.01076932864019653. feature name: AdjCloseSMA200_-8
10 importance: 0.010111141980689752. feature name: AdjCloseSMA50_to_AdjCloseSMA200_ratio_0
11 importance: 0.009991815876602228. feature name: AdjCloseSMA50_chg_-18
12 importance: 0.00992802567068378. feature n

104 importance: 0.0030585578359669965. feature name: AdjOpen_chg_-4
105 importance: 0.0030442576852830012. feature name: AdjCloseSMA50_to_AdjClose_ratio_-7
106 importance: 0.003029908681563042. feature name: AdjCloseSMA50_chg_-8
107 importance: 0.0029987366520998968. feature name: AdjCloseSMA200_chg_-9
108 importance: 0.002989017918459961. feature name: AdjCloseSMA50_-9
109 importance: 0.0029830862649936635. feature name: AdjCloseSMA10_-18
110 importance: 0.00297834111560445. feature name: AdjCloseSMA200_to_AdjClose_ratio_-1
111 importance: 0.0029744019479446423. feature name: AdjCloseSMA10_to_AdjCloseSMA50_ratio_-2
112 importance: 0.0029690045215502283. feature name: AdjCloseSMA50_to_AdjClose_ratio_-13
113 importance: 0.002957147433267521. feature name: AdjCloseRSI14_-17
114 importance: 0.0029046674584856237. feature name: AdjCloseSMA10_chg_-3
115 importance: 0.002904374620250145. feature name: AdjCloseRSI14_-15
116 importance: 0.0028936228947267035. feature name: AdjCloseSMA50_chg_-7

215 importance: 0.0014652017665944673. feature name: AdjCloseSMA50_chg_-1
216 importance: 0.0014623396381337544. feature name: AdjVolume_chg_-3
217 importance: 0.001455129007842726. feature name: AdjCloseSMA200_chg_-8
218 importance: 0.0014540029396754896. feature name: AdjClose_above_AdjCloseSMA50_-19
219 importance: 0.0014486989128927377. feature name: AdjClose_above_AdjCloseSMA10_-18
220 importance: 0.0014287667278006357. feature name: AdjCloseSMA50_to_AdjClose_ratio_-3
221 importance: 0.0014249658144024537. feature name: AdjClose_-7
222 importance: 0.0014116839787319548. feature name: AdjLow_chg_-9
223 importance: 0.0014033748349045346. feature name: AdjVolume_-15
224 importance: 0.001374395289062567. feature name: AdjClose_-1
225 importance: 0.0013486527046426117. feature name: AdjHigh_chg_-5
226 importance: 0.0013191790595020144. feature name: AdjCloseSMA200_-13
227 importance: 0.0013054251932043577. feature name: AdjCloseSMA10_to_AdjClose_ratio_-10
228 importance: 0.001304786684

327 importance: 0.0006129897337412172. feature name: AdjCloseSMA50_-12
328 importance: 0.0006106199181178581. feature name: AdjClose_chg_0
329 importance: 0.0006075702907117609. feature name: AdjOpen_chg_-19
330 importance: 0.0006059645784468765. feature name: AdjCloseSMA10_to_AdjCloseSMA50_ratio_0
331 importance: 0.0006059272631466694. feature name: AdjLow_-1
332 importance: 0.0006020913161249563. feature name: AdjCloseSMA200_chg_-5
333 importance: 0.000601932475993918. feature name: AdjCloseSMA50_chg_-9
334 importance: 0.0005957862884296887. feature name: AdjVolume_-16
335 importance: 0.0005934520320566795. feature name: AdjCloseRSI14_-4
336 importance: 0.0005931700145320073. feature name: AdjVolume_chg_-12
337 importance: 0.0005864622098702904. feature name: AdjCloseRSI14_-19
338 importance: 0.000582060947248811. feature name: AdjLow_-3
339 importance: 0.0005781227818209071. feature name: AdjOpen_chg_-3
340 importance: 0.0005658546099455603. feature name: AdjVolume_chg_-15
341 impor

453 importance: 0.0. feature name: AdjCloseSMA10_chg_-1
454 importance: 0.0. feature name: AdjHigh_-15
455 importance: 0.0. feature name: AdjOpen_0
456 importance: 0.0. feature name: AdjCloseSMA50_chg_-3
457 importance: 0.0. feature name: AdjOpen_-6
458 importance: 0.0. feature name: AdjCloseSMA200_chg_-11
459 importance: 0.0. feature name: AdjCloseSMA200_chg_-2
460 importance: 0.0. feature name: year
461 importance: 0.0. feature name: weekday
462 importance: 0.0. feature name: AdjOpen_-19
463 importance: 0.0. feature name: AdjVolume_0
464 importance: 0.0. feature name: AdjVolume_-2
465 importance: 0.0. feature name: AAPL.US
466 importance: 0.0. feature name: AdjVolume_-6
467 importance: 0.0. feature name: ABBV.US
468 importance: 0.0. feature name: AdjVolume_-10
469 importance: 0.0. feature name: AMZN.US
470 importance: 0.0. feature name: CSCO.US
471 importance: 0.0. feature name: GE.US
472 importance: 0.0. feature name: INTC.US
473 importance: 0.0. feature name: AdjClose_chg_-15
474 i

583 importance: 0.0. feature name: AdjCloseSMA200_above_AdjCloseSMA50_-17
584 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-19
585 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-18
586 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-17
587 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-16
588 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-15
589 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-14
590 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-13
591 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-12
592 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-11
593 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-10
594 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-9
595 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-8
596 importance: 0.0. feature name: AdjClose_above_AdjCloseSMA200_-7
597 importance: 0.0. feature nam

In [51]:
y_pred = cv.predict(X_test)

C:\Users\Lindstaedt\Anaconda3\envs\trader_env\lib\site-packages\sklearn\pipeline.py:331: DataConversionWarning: Data with input dtype bool, uint8, int64, float64 were all converted to float64 by StandardScaler.
  Xt = transform.transform(Xt)


In [52]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.76      0.77      0.76      1300
        True       0.32      0.32      0.32       456

   micro avg       0.65      0.65      0.65      1756
   macro avg       0.54      0.54      0.54      1756
weighted avg       0.65      0.65      0.65      1756



In [53]:
accuracy_score(y_test, y_pred)

0.6492027334851936

In [54]:
confusion_matrix(y_test, y_pred, labels=[False, True])

array([[996, 304],
       [312, 144]], dtype=int64)

In [82]:
y_pred

array([False, False, False, ..., False, False, False])

## next: add backtesting

In [89]:
# load the ticker list:
symbols = fastparquet.ParquetFile('../data/interim/symbols.parq').to_pandas().index.tolist()
    
print(symbols)

['AAPL.US', 'INTC.US', 'MSFT.US', 'AMZN.US', 'CSCO.US', 'NFLX.US', 'ABBV.US', 'GE.US']


In [90]:
# build dataframe with dates, stock tickers, real setup for profitable trade, and prediction
df_backtest = pd.concat([X_test.loc[:, symbols], y_test], axis=1)
df_backtest['setup_for_profitable_trade_prediction'] = y_pred
df_backtest.head()

,AAPL.US,INTC.US,MSFT.US,AMZN.US,CSCO.US,NFLX.US,ABBV.US,GE.US,setup_for_profitable_trade,setup_for_profitable_trade_prediction
Index,,,,,,,,,,
2017-05-10,0,0,0,0,0,0,1,0,False,False
2017-05-10,0,1,0,0,0,0,0,0,False,False
2017-05-10,0,0,1,0,0,0,0,0,False,False
2017-05-10,0,0,0,1,0,0,0,0,False,False
2017-05-11,0,0,1,0,0,0,0,0,False,False


In [88]:
# load the df_downloaded_all_ticker dataframe from feature engineering step:
df_downloaded_all_tickers = fastparquet.ParquetFile('../data/interim/df_downloaded_all_tickers.parq').to_pandas()
df_downloaded_all_tickers.head()

,AdjVolume,AdjOpen,AdjHigh,AdjLow,AdjClose,row_index,strategy_open_price,strategy_close_price,strategy_profit_dollars,strategy_profitability,setup_for_profitable_trade,ticker
Index,,,,,,,,,,,,
2013-11-13,49305200.0,68.853856,69.418776,68.715616,69.203974,219,69.493213,71.28764,1.794427,0.025822,True,GE.US
2013-11-13,49305200.0,68.853856,69.418776,68.715616,69.203974,219,69.493213,71.28764,1.794427,0.025822,True,CSCO.US
2013-11-13,49305200.0,68.853856,69.418776,68.715616,69.203974,219,69.493213,71.28764,1.794427,0.025822,True,MSFT.US
2013-11-13,49305200.0,68.853856,69.418776,68.715616,69.203974,219,69.493213,71.28764,1.794427,0.025822,True,NFLX.US
2013-11-13,49305200.0,68.853856,69.418776,68.715616,69.203974,219,69.493213,71.28764,1.794427,0.025822,True,INTC.US


In [92]:
for ticker in tqdm_notebook(symbols):
    print(ticker)

NameError: name 'tqdm_notebook' is not defined

## use TPOT